In [7]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, log_loss
import pandas as pd
from sklearn.model_selection import cross_val_score
import optuna
from pathlib import Path
import joblib

In [ ]:
gamesDF = pd.read_csv('./datasets/training_dataset.csv')
gamesDF = gamesDF.drop(columns=['HOME_L10_LOSSES', 'AWAY_L10_LOSSES', 'PERIOD', 'POINT_DIFF'])

In [9]:
X = gamesDF[['SECONDS_REMAINING','HOME_SCORE','AWAY_SCORE','HOME_WINS', 'HOME_LOSSES', 'AWAY_WINS', 'AWAY_LOSSES', 'HOME_L10_WINS', 'AWAY_L10_WINS']] 
y = gamesDF['HOME_WIN']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [10]:
def save_model(model, model_path_name):
    model_path = Path(model_path_name)
    joblib.dump(model, model_path)
    print(f"Model saved to {model_path.absolute()}")

X_tr, X_val, y_tr, y_val = train_test_split(
    X_train,
    y_train,
    test_size=0.2,
    stratify=y_train,
    random_state=42
)

def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 300, 800),
        "max_depth": trial.suggest_int("max_depth", 2, 3),
        "learning_rate": trial.suggest_float("learning_rate", 0.02, 0.05, log=True),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-2, 10.0, log=True),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-2, 10.0, log=True),
        "objective": "binary:logistic",
        "eval_metric": "logloss",
        "n_jobs": -1,
        "early_stopping_rounds": 50,
        "min_child_weight": trial.suggest_int("min_child_weight", 200, 300),
        "gamma": trial.suggest_float("gamma", 0.5, 5.0),
    }

    model = XGBClassifier(**params)
    model.fit(
        X_tr, y_tr,
        eval_set=[(X_val, y_val)],
        verbose=False,
    )
    y_proba = model.predict_proba(X_val)[:, 1]
    return log_loss(y_val, y_proba)

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=10)

print(study.best_params)

[I 2026-02-11 12:04:27,452] A new study created in memory with name: no-name-a6ce9972-7293-45ec-af2e-5a171832a331
[I 2026-02-11 12:05:08,312] Trial 0 finished with value: 0.474746821387863 and parameters: {'n_estimators': 687, 'max_depth': 2, 'learning_rate': 0.02851806938879716, 'subsample': 0.9977370244294912, 'colsample_bytree': 0.8053553948348537, 'reg_lambda': 3.972943439961816, 'reg_alpha': 2.759495093837856, 'min_child_weight': 242, 'gamma': 2.456355360571375}. Best is trial 0 with value: 0.474746821387863.
[I 2026-02-11 12:05:56,033] Trial 1 finished with value: 0.4596959175914084 and parameters: {'n_estimators': 665, 'max_depth': 3, 'learning_rate': 0.02026056101631898, 'subsample': 0.9581586458345048, 'colsample_bytree': 0.9497845015123165, 'reg_lambda': 0.170626814930273, 'reg_alpha': 0.012248432039889645, 'min_child_weight': 280, 'gamma': 0.6779560437997724}. Best is trial 1 with value: 0.4596959175914084.
[I 2026-02-11 12:06:25,432] Trial 2 finished with value: 0.451505117

{'n_estimators': 662, 'max_depth': 3, 'learning_rate': 0.04754917569113294, 'subsample': 0.7304181585723355, 'colsample_bytree': 0.7291461859424053, 'reg_lambda': 0.012387701742468959, 'reg_alpha': 1.2970527451941831, 'min_child_weight': 243, 'gamma': 1.7270113452917277}


In [11]:
bst = XGBClassifier(**study.best_params)
bst.fit(X_train, y_train)
save_model(bst, 'xgboost.joblib')


Model saved to /Users/lemons/Documents/universidad/cs/pj09-sports-betting/ml/xgboost.joblib


**Why probabilities are so extreme (95% / 5%)**

1. **XGBoost (and trees) are overconfident** – They output probabilities that get pushed toward 0 and 1. Good ranking, poor calibration.
2. **Your features are very informative** – Score + time left often makes the outcome almost certain, so the model is confident.
3. **Fix: calibrate** – Fit a calibrator (e.g. isotonic regression) on the model’s outputs so that when it says 70%, about 70% of those cases actually win. The cell below does this and saves the calibrated model for the backend.

**Optional training tweaks** (to get slightly softer raw probabilities before calibration): use **max_depth 2–3 only**, **higher min_child_weight** (e.g. 200–400), and **stronger reg_alpha/reg_lambda**. Calibration is still the main fix.

In [12]:
# Calibrate so probabilities are less extreme (e.g. 65/35 instead of 95/5)
from sklearn.calibration import CalibratedClassifierCV

calibrated = CalibratedClassifierCV(
    XGBClassifier(**study.best_params),
    method="isotonic",
    cv=5,
)
calibrated.fit(X_train, y_train)

# Compare raw vs calibrated on test set
p_raw = bst.predict_proba(X_test)[:, 1]
p_cal = calibrated.predict_proba(X_test)[:, 1]
print("Raw proba  (min, max):", round(p_raw.min(), 3), round(p_raw.max(), 3))
print("Calibrated (min, max):", round(p_cal.min(), 3), round(p_cal.max(), 3))

save_model(calibrated, "xgboost_calibrated.joblib")
print("Calibrated model saved. Restart backend to use it.")

Raw proba  (min, max): 0.0 1.0
Calibrated (min, max): 0.0 1.0
Model saved to /Users/lemons/Documents/universidad/cs/pj09-sports-betting/ml/xgboost_calibrated.joblib
Calibrated model saved. Restart backend to use it.
